In [1]:
#! wget -r -e robots=off -P ../ --no-parent  --no-host-directories --reject="index.html*" --convert-links http://tablebase.lichess.ovh/tables/standard/3-4-5/

In [2]:
import chess
from chess import syzygy
from board.BoardEncoder import BoardEncoder
import numpy as np
from montecarlo.MonteCarloNode import MonteCarloNode
import pickle
import os
from datetime import datetime
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
tablebase = syzygy.Tablebase()
tablebase.add_directory('../tables/standard/3-4-5')

290

## Probing tablebase for selected positions

We are using Syzygy tablebases. More information here: https://python-chess.readthedocs.io/en/latest/syzygy.html

In [4]:
def print_dtz(fen):
    dtz = tablebase.probe_dtz(chess.Board(fen))
    print(f"Distance to checkmate or a zeroing move: {dtz}")


White to move:

![](https://fen2image.chessvision.ai/8/8/8/8/3k4/8/3K4/3Q4_w_-_-_0_1?color=white)


In [5]:
print_dtz('8/8/8/8/3k4/8/3K4/3Q4 w - - 0 1')

Distance to checkmate or a zeroing move: 13


A positive value for DTZ indicates that the side to move needs 13 moves to win (or zero the 50-move counter), so here white is winning.


White to move:

![](https://fen2image.chessvision.ai/k7/p7/8/K7/8/8/8/8_w_-_-_0_1?color=white)


In [6]:
print_dtz('k7/p7/8/K7/8/8/8/8 w - - 0 1')

Distance to checkmate or a zeroing move: 0


Zero means that the position is drawn.

White to move:

![](https://fen2image.chessvision.ai/8/R7/3r4/8/4K3/8/4p3/4k3%20w%20-%20-%200%201?color=white)

In [7]:
print_dtz('8/R7/3r4/8/4K3/8/4p3/4k3 w - - 0 1')

Distance to checkmate or a zeroing move: -10


A negative value indicates that the side to play will lose in 10 moves, assuming the best play by both sides.

In [8]:
def dtz_to_eval(dtz):
    if dtz > 0:
        return max(101 - dtz, 50)
    elif dtz < 0:
        return min(-101-dtz, -50)
    return 0

## Building a training dataset

In [9]:
def bitarray_to_ndarray(bitarray):
    return np.array([int(bit) for bit in bitarray], dtype=bool)

def put_piece(board, color, piece, check_validity=True):
    while True:
        square = np.random.randint(0, 64)
        while board.piece_at(square) is not None:
            square = np.random.randint(0, 64)
        board.set_piece_at(square, chess.Piece(piece, color)) 
        if board.is_valid() or not check_validity:
            break
        else:
            board.remove_piece_at(square)

def generate_random_fen():
    # Generate a random position, where
    # for each side there is a king and optionally a queen and/or a rook and/or a pawn
    
    board = chess.Board()
    board.clear_board()
    board.castling_rights = 0
    board.turn = chess.WHITE if np.random.randint(0, 2) == 0 else chess.BLACK

    # Put kings first because positions without them are illegal
    put_piece(board, chess.WHITE, chess.KING, check_validity=False)
    put_piece(board, chess.BLACK, chess.KING, check_validity=True)

    # Put remaining pieces
    while True:
        color_piece_count = {
            chess.WHITE: {
                chess.QUEEN: np.random.randint(0, 2),
                chess.ROOK: np.random.randint(0, 2),
            },
            chess.BLACK: {
                chess.QUEEN: np.random.randint(0, 2),
                chess.ROOK: np.random.randint(0, 2),
            }
        }
        if sum(color_piece_count[chess.WHITE].values()) + sum(color_piece_count[chess.BLACK].values()) <= 3: # We can have up to 5 pieces, kings included
            break

    for color in chess.COLORS:
        for piece, count in color_piece_count[color].items():
            for _ in range(count):
                put_piece(board, color, piece)
    
    return board.fen()
                

N_SAMPLES = 100000

X_rows = []
evals = []
fens = []

# for _ in range(N_SAMPLES):
#     fen = generate_random_fen()
#     fens.append(fen)
    
#     board = chess.Board(fen)
    
#     X_row = bitarray_to_ndarray(BoardEncoder.encode(board))
#     X_rows.append(X_row)

#     dtz = tablebase.probe_dtz(board)
    
#     evals.append(dtz_to_eval(dtz))

X = np.array(X_rows)
y = np.array(evals)  

In [10]:
X.shape

(0,)

In [11]:
y.shape

(0,)

## Saving the model

In [12]:
# path = f'../models/model-{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl'
# os.mknod(path)

# with open(path, 'ab') as file:
#     pickle.dump(reg, file)

## Loading the model

In [13]:
load_path = '../models/model-20231218192512.pkl'
reg: MLPRegressor
with open(load_path, 'rb') as file:
    reg = pickle.load(file)

## Crossvalidation to see if MLP regressor is a good choice

In [14]:
# reg = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1000, verbose=True, alpha=0.5)

# cv_score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_absolute_error')

# print(f"Crossvalidation scores: {cv_score}")
# print(f"Mean: {cv_score.mean()}")
# print(f"Std: {cv_score.std()}")

## Proper training

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# reg.fit(X_train, y_train)
# mse = reg.score(X_test, y_test)

# print(f"Mean squared error: {mse}")

## Model predictions

In [16]:
def get_estimation(reg, fen):
    board = chess.Board(fen)
    X = bitarray_to_ndarray(BoardEncoder.encode(board)).reshape(1, -1)
    return reg.predict(X)[0]


White to move:

![](https://fen2image.chessvision.ai/8/8/8/8/3k4/8/3K4/3Q4_w_-_-_0_1?color=white)

In [17]:
get_estimation(reg, '8/8/8/8/3k4/8/3K4/3Q4 w - - 0 1')

88.80806033383806


White to move:

![](https://fen2image.chessvision.ai/k7/p7/8/K7/8/8/8/8_w_-_-_0_1?color=white)

In [18]:
get_estimation(reg, 'k7/p7/8/K7/8/8/8/8 w - - 0 1')

0.6293887017390813

White to move:

![](https://fen2image.chessvision.ai/8/R7/3r4/8/4K3/8/4p3/4k3%20w%20-%20-%200%201?color=white)

In [19]:
get_estimation(reg, '8/R7/3r4/8/4K3/8/4p3/4k3 w - - 0 1')

3.612007627864023

## Monte Carlo Tree Search

In [20]:
def get_next_move(board):
    mc_node = MonteCarloNode(root_board=board,
                             is_leaf=board.is_game_over(),
                             parent=None,
                             node_board=board, 
                             move_stack=[],
                             evaluation_func=lambda board: get_estimation(reg, board.fen()))
    mc_node.explore()
    

White to move:

![](https://fen2image.chessvision.ai/8/R7/3r4/8/4K3/8/4p3/4k3%20w%20-%20-%200%201?color=white)

In [21]:
board = chess.Board(fen='8/R7/3r4/8/4K3/8/4p3/4k3 w - - 0 1')

mc_node = MonteCarloNode(root_board=board,
                         is_leaf=board.is_game_over(),
                         parent=None,
                         node_board=board, 
                         move_stack=[],
                         evaluation_func=lambda board: get_estimation(reg, board.fen()))

for i in range(100):
    mc_node.explore()

print(mc_node.next())

(Move.from_uci('a7a8'), <montecarlo.MonteCarloNode.MonteCarloNode object at 0x7f5dd09df210>)
